In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

##Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist',with_info= True, as_supervised= True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']
num_validation_samples = 0.1* mnist_info.splits['train'].num_examples

#num_validation_samples can be float, to convert to int
num_validation_samples= tf.cast(num_validation_samples,tf.int64)


num_test_samples = 0.1* mnist_info.splits['test'].num_examples
num_test_samples= tf.cast(num_test_samples,tf.int64)


#for scaler rep
def scale(image,label):
  image = tf.cast(image,tf.float32)
  image /= 255.
  return image,label

#scale train and test data
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)


#shuffle data & create validation dataset
BUFFER_SIZE = 10000  #for a large data set defines no.of records to consider at once
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

#extract validation data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

#remove validation data from training sample
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

#Batching data
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
#take all validation and test data as a single batch
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
validation_inputs, validation_targets = next(iter(validation_data))

##Model

###Outlining Model

In [4]:
input_size=784
output_size=10
hidden_layer_size = 50

model = tf.keras.Sequential([
                              #transforms/flattens tensor into vectors
                              tf.keras.layers.Flatten(input_shape=(28,28,1)),
                              #middle layers
                              tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                              tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                              #output layer
                              tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

###Choose Optimizer and Loss Function

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

###Training

In [6]:
NUM_EPOCHS = 5
model.fit(train_data,epochs=NUM_EPOCHS, validation_data= (validation_inputs,validation_targets), verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.4162 - accuracy: 0.8819 - val_loss: 0.2178 - val_accuracy: 0.9387 - 8s/epoch - 16ms/step
Epoch 2/5
540/540 - 4s - loss: 0.1838 - accuracy: 0.9468 - val_loss: 0.1634 - val_accuracy: 0.9525 - 4s/epoch - 7ms/step
Epoch 3/5
540/540 - 5s - loss: 0.1432 - accuracy: 0.9579 - val_loss: 0.1311 - val_accuracy: 0.9607 - 5s/epoch - 9ms/step
Epoch 4/5
540/540 - 4s - loss: 0.1171 - accuracy: 0.9649 - val_loss: 0.1042 - val_accuracy: 0.9680 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0987 - accuracy: 0.9704 - val_loss: 0.0886 - val_accuracy: 0.9722 - 4s/epoch - 7ms/step


###Test the Model

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 0s 33ms/step - loss: 0.1066 - accuracy: 0.9679


In [9]:
print('Test Loss: {0:.2f}. Test Accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test Loss: 0.11. Test Accuracy: 96.79%
